# Run a Gene Set Enrichment Analysis using Reactome_2016

This notebook looks at the significant genes in at least one cancer. GSEA is used to recognize pathways with multiple significant genes. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(df.Proteomics.unique()) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.head(5)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Processing of Capped Intron-Containing Pre-mRN...,116/193,1.379053e-53,2.109952e-50,0,0,4.603878,560.363828,NUP107;NUP188;EIF4A3;HNRNPU;EFTUD2;SNRPD2;SNRP...,Reactome_2016
1,Gene Expression Homo sapiens R-HSA-74160,434/1631,1.734808e-53,1.327128e-50,0,0,2.038257,247.619892,TDRKH;RPL4;ATF2;MDC1;NUP107;HNRNPU;EHMT1;PHAX;...,Reactome_2016
2,mRNA Splicing - Major Pathway Homo sapiens R-H...,85/134,3.384052e-42,1.725866e-39,0,0,4.858892,463.973174,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
3,mRNA Splicing Homo sapiens R-HSA-72172,85/144,1.081367e-38,4.136230e-36,0,0,4.521469,395.266853,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
4,Infectious disease Homo sapiens R-HSA-5663205,123/348,8.726101e-27,2.670187e-24,0,0,2.707379,162.452135,RPL4;NUP107;NUP188;RPL10A;RPL9;PSMD8;RPS15;PSM...,Reactome_2016


# Run GSEA for significant genes in multiple cancers

In [4]:
mult_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_df.Proteomics.unique()) # list of genes with a sig pval in >= 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [5]:
enr2.res2d.head(10)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Gene Expression Homo sapiens R-HSA-74160,74/1631,2.081067e-17,3.184033e-14,0,0,2.917745,112.073703,WDR3;WDR4;PTEN;HNRNPU;NAT10;ZC3H8;ADAR;PPP2R2A...,Reactome_2016
1,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",37/462,2.789117e-16,2.133675e-13,0,0,5.150262,184.459924,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
2,Cell Cycle Homo sapiens R-HSA-1640170,38/566,3.204908e-14,1.634503e-11,0,0,4.317544,134.152603,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
3,Processing of Capped Intron-Containing Pre-mRN...,23/193,3.858260e-14,1.475784e-11,0,0,7.663729,236.701740,RANBP2;DDX5;NUP210;CPSF1;NUP155;NCBP1;NUP133;D...,Reactome_2016
4,DNA strand elongation Homo sapiens R-HSA-69190,11/32,1.039399e-12,3.180562e-10,0,0,22.106109,609.960125,GINS2;RFC3;PCNA;RFC4;MCM7;RFC2;GINS4;MCM3;MCM4...,Reactome_2016
5,rRNA modification in the nucleus Homo sapiens ...,13/58,4.077700e-12,1.039814e-09,0,0,14.414015,378.014573,UTP6;IMP3;WDR3;HEATR1;NAT10;WDR75;IMP4;PWP2;WD...,Reactome_2016
6,Transport of Mature Transcript to Cytoplasm Ho...,14/74,7.117502e-12,1.555682e-09,0,0,12.166507,312.295560,RANBP2;NCBP1;NUP210;NUP133;CPSF1;NUP155;CPSF3;...,Reactome_2016
7,Unwinding of DNA Homo sapiens R-HSA-176974,7/12,1.522584e-10,2.911941e-08,0,0,37.513398,848.006944,GINS2;MCM7;GINS4;MCM3;MCM4;MCM5;MCM2,Reactome_2016
8,Transport of Mature mRNA Derived from an Intro...,10/37,1.718728e-10,2.921837e-08,0,0,17.380725,390.792850,RANBP2;CPSF1;NCBP1;NUP210;NUP133;NUP155;NUP85;...,Reactome_2016
9,Transport of Mature mRNAs Derived from Intronl...,10/38,2.300707e-10,3.520081e-08,0,0,16.923337,375.573441,RANBP2;CPSF1;NCBP1;NUP210;NUP133;NUP155;NUP85;...,Reactome_2016
